In [ ]:
import uproot3 as uproot
import numpy as np
import awkward as ak

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
filename = ""
file = uproot.open(filename)
file

In [ ]:
file.keys()

In [ ]:
def rebin(array, time): 
    m, n = array.shape 
    N = n/time 
    if N==int(N):
        N=int(N)
        rebinned = np.array([np.sum(array[:,time*k:time*k+time], axis =1).reshape(m, -1)/time for k in range(N)])
        return rebinned.reshape(N,-1).T
    else: return "error binning"

In [ ]:
def plot_stack(class_softmax):
    filename = ""
    file = uproot.open(filename)
    legends = [str(key)[2:-3] for key in file.keys() if key not in (b'purity_enu;1', b'efficiency_enu;1', b'selected_events;1')]
    data = np.array([file[key].numpy()[0] for key in file.keys() if key not in (b'purity_enu;1', b'efficiency_enu;1', b'selected_events;1')])
    bins = np.array([file[key].numpy()[1] for key in file.keys() if key not in (b'purity_enu;1', b'efficiency_enu;1', b'selected_events;1')])
    centers = bins[:, :-1]+np.diff(bins)/2
    fig, ax = plt.subplots(figsize = (8,6))
    ax.hist(list(centers), bins=bins[0], weights=list(data), stacked=True, fill=True)
    ax.legend(legends)
    ax.set_title("class "+ class_softmax)
    #if class_softmax=='mu':
        #ax.set_ylim([0,100])
    return fig, ax

In [ ]:
for el in ['e','mu','gamma']:
    A = plot_stack(el)

In [ ]:
def plot_not_stack(class_softmax, rebin_time):
    filename = "output_1ringe_analysis_prob_"+class_softmax+ ".root"
    file = uproot.open(filename)
    legends = [str(key)[2:-3] for key in file.keys() if key not in (b'purity_enu;1', b'efficiency_enu;1', b'selected_events;1')]
    data = np.array([file[key].numpy()[0] for key in file.keys() if key not in (b'purity_enu;1', b'efficiency_enu;1', b'selected_events;1')])
    bins = np.array([file[key].numpy()[1] for key in file.keys() if key not in (b'purity_enu;1', b'efficiency_enu;1', b'selected_events;1')])
    centers = bins[:, :-1]+np.diff(bins)/2
    
    components = len(legends)
    
    normed_data = data/np.sum(data, axis=1).reshape(data.shape[0],1)
    
    
    normed_data = rebin(normed_data, rebin_time)
    centers = rebin(centers, rebin_time)
    d = np.diff(centers)[:, 0].reshape(components, 1)
    bins = np.concatenate([np.zeros((components,1)), centers+d/2], axis=1)
    
    Dots_plot = False
    if Dots_plot:
        fig, ax = plt.subplots(figsize = (16,12))   
        for component in range(components):
            plt.plot(centers[component], normed_data[component], 'o-')
        plt.legend(legends)
        plt.title("class "+class_softmax)
        plt.savefig('true_comp_'+class_softmax+'_'+str(int(50/rebin_time))+'_bins.pdf')
    
    histo_plot =True
    
    if histo_plot:
        fig, ax = plt.subplots(figsize = (16,12))

        for component in range(components):
            ax.hist(centers[0], bins=bins[0], weights=normed_data[component], label = legends[component], histtype = 'step', linewidth=1.5)
        ax.legend()

In [ ]:
for el in ['e','mu','gamma']:
    B = plot_not_stack(el, 5)